## White noise localization
### Extension 1 : concatenate the data of two mics

In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt 
import wave 
from scipy.io import wavfile
import soundfile
import time 
from scipy import signal
import logging
import datetime
from itertools import combinations
from Processing import calculate_angle_error

In [ ]:
path = 'data/kemar_h_theta_1deg_time.npy' #  the path to the file
path_ = 'data/lego1_h_theta_time.npy'
hrir  = np.load(path) #head related transfer function
T,D,M = hrir.shape
#hrir = np.random.randn(T,D,M)

fs = 16000 #samples/second
T_ =0.5 #length of signal in time domain 
N = int(T_*fs) #number of samples of the signal 

mean = 0
std = 1 

hrir_tot = np.zeros((320, 360))
# mic0 + mic1
hrir_tot = np.concatenate((hrir[:,:,0], hrir[:,:,1]),axis=0)
obs_len = N + int(hrir_tot.shape[0]) -1  #length of the convolution


J=2
Df = int(hrir.shape[1]) #directions 

avg_err = 0
perfect = 0
SNR = 30
runs = 100
J=2
for rns in range(runs):
    St_all = np.zeros((N, J))

    for j in range(J):
        St_all[:, j] = np.random.normal(mean, std, size=N) #white noise

        
    theta = np.random.choice(range(Df), J, replace=False) #choose the directions randomly on the grid   
    yt = np.zeros((obs_len, )) #recorded time domain signal  voir deuxième paramètre 


    

    for j in range(J):
        yt += np.convolve(St_all[:, j], hrir_tot[:, theta[j]]) #source signal convolved with corresponding directional response

    # Generate noise at required SNR    
    sig_norm = np.linalg.norm(yt)
    noise_t = np.random.randn(obs_len, ) #additive gaussian noise
    noise_norm = sig_norm/(10.**(SNR/20.))
    noise_t = noise_norm*noise_t/np.linalg.norm(noise_t)

    yt += noise_t #noisy signal
    f, t, y = signal.stft(yt, fs=1.0, window='hann', nperseg=320)
    N_ = y.shape[1] #number of frames

    y_mean = np.mean(np.abs(y)**2, axis=1) #mean power frame
    y_mean = y_mean/np.linalg.norm(y_mean) #normalize the observation

    
    anglesf = np.arange(Df, dtype=np.int64)*360./Df # list of angles in degrees
    # Initialize variables
    best_ind = np.inf #index corresponding to best direction tuple
    smallest_norm = np.inf #smallest projection error
    best_dir = theta #best direction tuple
    H = np.fft.rfft(hrir_tot,axis=0)
    
    
    # Search all combinations
    pairs2 = combinations(range(D), J) #all combinations in range(D) and length J=2
    for q2, d2 in enumerate(pairs2): 
        Bj = np.abs(H[:, d2])**2 #vectors in current guess contains the two vectors 
        Pj = Bj.dot(np.linalg.pinv(Bj)) #projection matrix
        proj_error = np.linalg.norm((np.eye(int(y.shape[0])) - Pj).dot(y_mean)) #projection error

        if proj_error <= smallest_norm:
            smallest_norm = proj_error
            best_ind = q2
            best_dir = d2
    step=1
    theta_hat = step*np.array(best_dir) #map coarse index to fine index
    min_err, best_perm = calculate_angle_error(theta, theta_hat, anglesf) #calculate error between chosen and true directions        
    if min_err==0:
        perfect+=1   
        
    avg_err += min_err 
    print('iteration ' + str(rns+1) +' error : '+ str(min_err) + '  ground truth : ' + str(theta)+ '  estimation : ' + str(best_perm) )

avg_err = avg_err/runs
print('average error : ' + str(avg_err)+' acc   '+str(perfect/runs))